<a href="https://colab.research.google.com/github/TE-Raven/python-project/blob/main/part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import moduls

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from gensim import utils
import gensim.parsing.preprocessing as gsp
import nltk
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import sklearn
import operator
drive.mount('/content/drive')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


dataprocessing

In [ ]:
root=r"/content/drive/MyDrive/news_classifacation/bbc"
filename=[]
dirs=os.listdir(root)
mark=0
data= pd.DataFrame(columns=('text','mark'))
#print(dirs)
for dir in dirs:
    dir_path = root +'/' +dir
    names = os.listdir(dir_path)
    if dir == 'business':
        mark =0
    if dir == 'entertainment':
        mark =1
    if dir == 'politics':
        mark =2
    if dir == 'sport':
        mark =3
    if dir == 'tech':
        mark =4
    for n in names:
     filename.append(dir_path + '/'+ n )
    for m in filename:
        with open(m, encoding='gb18030', errors='ignore') as content:
            # text=content.rstrip('\n')
            a = content.read()
            b = a.replace('\n', '').replace('\r', '')
            data1 = pd.DataFrame({'text': b, "mark": mark}, index=[0])
            data = data.append(data1, ignore_index=True)
    filename=[]
#data.to_csv("dataset.csv") this command can generate a dataset file for check or other using


In [ ]:
data1=data[['text','mark']]
lable=data['mark']

set filters

In [ ]:
filters = [
               gsp.strip_punctuation,          
               gsp.strip_multiple_whitespaces,  
               gsp.strip_numeric,              
               gsp.remove_stopwords,           
               gsp.strip_short,               
               gsp.stem_text                    
      ]

data cleaning

In [ ]:
data=pd.DataFrame(columns=(['text']))
for i in range(0,len(data1)):
    a=data1.iloc[i]['text']
    a=a.lower()
    a=utils.to_unicode(a)
    for f in filters:
        a=f(a)
    d = pd.DataFrame({'text': a}, index=[0])
    data = data.append(d, ignore_index=True)

data['mark']=label

data_set_spilt

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data['text'],data['mark'],test_size=0.2 , stratify=data['mark'],random_state=389 )

word frequency

In [ ]:
#get feature
lemmatizer = nltk.stem.WordNetLemmatizer()

def get_list_tokens(string):
  sentence_split=nltk.tokenize.sent_tokenize(string)
  list_tokens=[]
  for sentence in sentence_split:
    list_tokens_sentence=nltk.tokenize.word_tokenize(sentence)
    for token in list_tokens_sentence:
      list_tokens.append(lemmatizer.lemmatize(token).lower())
  return list_tokens

stopwords=set(nltk.corpus.stopwords.words('english'))
stopwords.add(".")
stopwords.add(",")
stopwords.add("--")
stopwords.add("``")

dict_word_frequency={}

for i in x_train:
    sentence_tokens = get_list_tokens(i)
    for word in sentence_tokens:
        if word in stopwords: continue
        if word not in dict_word_frequency:
            dict_word_frequency[word] = 1
        else:
            dict_word_frequency[word] += 1
sorted_list = sorted(dict_word_frequency.items(), key=operator.itemgetter(1), reverse=True)[:1000]

vocabulary=[]
for word,frequency in sorted_list:
  vocabulary.append(word)


# transform sentences into vectors
def get_vector_text(list_vocab,string):
  vector_text=np.zeros(len(list_vocab))
  list_tokens_string=get_list_tokens(string)
  for i, word in enumerate(list_vocab):
    if word in list_tokens_string:
      vector_text[i]=list_tokens_string.count(word)
  return vector_text

df_train=[]
df_test=[]
x_test_1=[]
for i in x_train:
    vector_review = get_vector_text(vocabulary, i)
    df_train.append(vector_review)
for i in x_test:
    vector_review = get_vector_text(vocabulary, i)
    df_test.append(vector_review)
for i in x_test:
    vector_review = get_vector_text(vocabulary, i)
    x_test_1.append(vector_review)
X_train_sentanalysis=np.asarray(df_train)
Y_train_sentanalysis=np.asarray(y_train)
x_test_1_sentanalysis=np.asarray(x_test_1)

fs_sentanalysis=SelectKBest(chi2,k=500).fit(X_train_sentanalysis,Y_train_sentanalysis)
X_train_sentanalysis_new=fs_sentanalysis.transform(X_train_sentanalysis)
x_test_sentanalysis_new=fs_sentanalysis.transform(x_test_1_sentanalysis)

svm_clf_sentanalysis=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf_sentanalysis.fit(X_train_sentanalysis_new,Y_train_sentanalysis)
print('word frequency')
print(classification_report(y_test,svm_clf_sentanalysis.predict(x_test_sentanalysis_new)))

word frequency
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       102
           1       0.90      0.97      0.94        77
           2       0.96      0.90      0.93        84
           3       0.96      0.99      0.98       102
           4       0.99      0.95      0.97        80

    accuracy                           0.95       445
   macro avg       0.95      0.95      0.95       445
weighted avg       0.95      0.95      0.95       445



TFIDF

In [ ]:
tfidf=TfidfVectorizer(ngram_range=(1,2),max_features=20,stop_words=['english'],max_df=0.6)
x_train_new=tfidf.fit_transform(x_train)
x_test_new=tfidf.fit_transform(x_test)
#feature selection
fs_sentanalysis=SelectKBest(chi2,k=10).fit(x_train_new,y_train)
x_train_1=fs_sentanalysis.transform(x_train_new)
x_test_1=fs_sentanalysis.transform(x_test_new)
svm_clf_sentanalysis=sklearn.svm.SVC(kernel="linear",gamma='auto')
svm_clf_sentanalysis.fit(x_train_new,y_train)
print(classification_report(y_test,svm_clf_sentanalysis.predict(x_test_new)))

              precision    recall  f1-score   support

           0       0.53      0.59      0.56       102
           1       0.48      0.53      0.51        77
           2       0.57      0.38      0.46        84
           3       0.68      0.72      0.70       102
           4       0.44      0.46      0.45        80

    accuracy                           0.55       445
   macro avg       0.54      0.54      0.53       445
weighted avg       0.55      0.55      0.54       445



bio_gram

In [ ]:
sel = SelectKBest(chi2, k=3000)
pipe=Pipeline([('vec',CountVectorizer(ngram_range=(2,2),min_df=3,max_df=0.9,max_features=4500)),
               ('sel',sel),
               ('clf',LogisticRegression(C=4,dual=False,max_iter=10000))])
clf=pipe.fit(x_train,y_train)
print('bio_gram')
print(classification_report(y_test,clf.predict(x_test)))

bio_gram
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       102
           1       0.85      0.94      0.89        77
           2       0.91      0.87      0.89        84
           3       0.91      0.97      0.94       102
           4       0.94      0.82      0.88        80

    accuracy                           0.90       445
   macro avg       0.90      0.90      0.90       445
weighted avg       0.91      0.90      0.90       445



tri_gram

In [ ]:
sel=SelectKBest(chi2,k=2500)
pipe=Pipeline([('vec',CountVectorizer(ngram_range=(3,3),min_df=3,max_df=0.9,max_features=3000)),
        ('sel',sel),
        ('clf',LogisticRegression(C=4,dual=False,max_iter=10000))])
clf=pipe.fit(x_train,y_train)
print('tri_gram')
print(classification_report(y_test,clf.predict(x_test)))

tri_gram
              precision    recall  f1-score   support

           0       0.95      0.57      0.71       102
           1       0.38      0.97      0.55        77
           2       0.97      0.69      0.81        84
           3       0.99      0.68      0.80       102
           4       0.91      0.66      0.77        80

    accuracy                           0.70       445
   macro avg       0.84      0.71      0.73       445
weighted avg       0.86      0.70      0.73       445

